The following are the libraries and helper functions used in this notebook.
Unless interested in what's going on under the hood, this section can be skipped:

In [27]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression, LogisticRegression


def fit_lm_and_return_plot(data, logistic=False, show_decision_line=False):
    """
    generate plot of linear / logistic regression model with decision boundry

    :param data: (pandas DataFrame) example data to generate plot
    :param logistic: (bool) set True for Logistic Regression else Linear
    :param show_decision_line: (bool) set True to show p = 0.5 decision boundary
    :return:
    """

    # define and fit model
    if logistic:
        clf = LogisticRegression()
    else:
        clf = LinearRegression()
    clf.fit(data["x"].values.reshape(-1,1),
            data["y"])

    # plot data on scatter plot
    y_labels = data["y"].apply(lambda y: "y = 1" if y else "y = 0")
    fig = px.scatter(data,
                     x="x",
                     y="y",
                     color=y_labels)

    # add y predictions from model to plot as line
    x_values = np.arange(0, 10, 0.1)
    if logistic:
        y_pred_values = clf.predict_proba(x_values.reshape(-1,1))[:,1]
    else:
        y_pred_values = clf.predict(x_values.reshape(-1, 1))
    fig.add_trace(
        go.Scatter(
            x=x_values,
            y=y_pred_values,
            mode="lines",
            line=go.scatter.Line(color="gray"),
            showlegend=False)
    )

    # add p = 0.5 decision line to plot if required
    if show_decision_line:
        if logistic:
            decision_value = float(-clf.intercept_/clf.coef_)
        else:
            decision_value = float((0.5 - clf.intercept_)/ clf.coef_)
        x_decision_values = [decision_value] * 2
        y_decision_values = [-.1, 1.1]
        fig.add_trace(
            go.Scatter(
                x=x_decision_values,
                y=y_decision_values,
                mode="lines",
                line=go.scatter.Line(color="red", dash="dash"),
                showlegend=False)
        )
    fig.update_yaxes(range=[-.1, 1.1])
    fig.update_xaxes(range=[-1, 10.5])
    return fig

eg_data = {
    "x": np.concatenate([np.random.normal(2, 1, 20),
                         np.random.normal(4, 0.5, 20)]),
    "y": np.concatenate([np.zeros(20),
                         np.ones(20)])
}
eg_df = pd.DataFrame(eg_data)

## Logistic Regression

This notebooks is the first of three "deep-dives" into the most successful of
 the ACE models trained so far. We'll begin with logistic regression as
 conceptually it is by far the most simple of these models. We'll get the
 theory out of the way first:

### The Theory

Logistic regression is very similar to the linear regression - indeed,
we can quickly explain it by comparing the two.

Linear regression attempts to directly predict a continuous output by
computing a "line of best fit" - most optimal the solution to the following
formula:

$y =  \beta_{0} + \beta_{1}x_{1} + \beta_{2}x_{2} + ... + \beta_{n}x_{n} + \epsilon$

which is a linear combination of the explanatory variables ($x_{i}$'s) and model
coefficients / weights ($\beta_{i}$'s) that best fits the outcome variable /
label ($y$), with an error term $\epsilon$ representing the inevitable
deviations from this line.

It's a fantastic tool for prediction tasks as it's simple, doesn't require
lots of data, and the output is easy to interpret. If we want to understand
the effect one of our variables has on the output, say $x_{3}$, we need simply
look at the coefficient associated with it, $\beta_{3}$ - increasing $x_{3}$ by
one unit increases the output prediction $y_{i}$ by the value of $\beta_{3}$
 (holding all other variables constant).

However, linear regression is not well suited to classification tasks. If we
were to fit a linear
regression model to a
really simple classification
 task, the solution might look like this:

In [28]:
lin_reg_plot = fit_lm_and_return_plot(eg_df)
lin_reg_plot.show()

This is obviously not an optimal solution to the problem. Our true y's are
all zeros and ones, but the linear model predicts zero / one
at single, very specific, points and is wrong everywhere else - Particularly
so at the extremes where the classification should be most obvious!

You might argue that we could simply classify those predictions above 0.5 as
 one and the others zero. This can be visualised by adding a decision
 boundary to the above plot:

In [29]:
lin_reg_plot = fit_lm_and_return_plot(eg_df,
                                      show_decision_line=True)
lin_reg_plot.show()

This may now seem like a reasonable solution - the model chooses a decent
decision boundary. But consider the following:

The examples that lie at the extremes of our classification groups (in this
case those x values approaching 0 or 6) shouldn't sway our decisions - they
are easy to classify and don't contribute to the confusion where the classes
overlap. But, if we add a number of examples to the extremes of one of the
groups, in this case we'll add some extra examples classified as y = 1, we see
the following:

In [30]:
extra_values = pd.DataFrame(
    {"x": np.random.normal(8, 1, 10),
     "y": np.ones(10)}
)
eg_df_ext = eg_df.append(extra_values)

In [31]:
lin_reg_plot_ext = fit_lm_and_return_plot(eg_df_ext,
                                          show_decision_line=True)
lin_reg_plot_ext.show()

The linear regression model's predictions change dramatically and it moves it's
boundary towards these new extreme values, which is obviously not desirable.

A better solution, accounting for all of the above issues, is to tweak the
linear regression function to be better suited to the classification task.
Instead of directly outputting numeric predictions of y, we re-formulate
the model function so that it outputs probabilities that $y = 1$. To do
this, we'll first take the output of the same linear regression formula we
had before, and we'll rename the output $\theta$ to avoid
 confusing it with a prediction for $y$:

$\theta =  \beta_{0} + \beta_{1}x_{1} + \beta_{2}x_{2} + ... + \beta_{n}x_{n} +
\epsilon$

Then we use the $\theta$ value to calculate our probability that $y = 1$
output using a "link function" that scales $\theta_{i}$ to between zero and one:

$p(y = 1 | \boldsymbol x) = \frac{1}{1 + e^{-\theta}} $

The result is model predictions that look like this:

In [32]:
log_reg_plot = fit_lm_and_return_plot(eg_df,
                                      logistic=True,
                                      show_decision_line=True)
log_reg_plot.show()

You can see that, after adjusting the model, the predictions are all
probabilites - they lie very close to zero or one at the extremes and then sharply curve as they approach the decision boundary. These
 predictions seem to do a much better job of fitting to the data. Moreover,
 if we add the same additional extreme values as before the model's predictions and decision boundary remains unmoved:

In [33]:
log_reg_plot_ext = fit_lm_and_return_plot(eg_df_ext,
                                          logistic=True,
                                          show_decision_line=True)
log_reg_plot_ext.show()

### Fitting to ACE Data

Great! With that done we can move on to fitting the model to the ACE dataset.
 We'll quickly run a cross validation parameter search to find the best
 performing Logistic Regression model and its respective scores:

In [8]:
import sys
sys.path.append('/Users/samrelins/Documents/LIDA/ace_project/')
from src.data_prep import *
from src.train_test import *
from IPython.display import display, HTML

data_loc = "/Users/samrelins/Documents/LIDA/ace_project/data/ace_data_extra.csv"
ace_data_orig = pd.read_csv(data_loc)
ace_data_orig.hospital_reqd.apply(
    lambda x: 0 if x == "Y" else 1
).sum()

421

In [6]:
ace_data_orig

,hospital_reqd,referral_from,referral_profession,age,address,ethnicity,gender,allergies,referral_date,referral_time,...,gut_feeling,ox_sat,resp_rate,heart_rate,temp,sepsis,safeguarding,medical_history,examination_summary,recommendation
0,N,CCDA,Consultant,8,BD07,Indian,F,NKA,Winter,Morning,...,low concern,97,20,118,36.5,None noted,N,3 day history of cough and wheeze. Known asthm...,"Looks well, running around talking in sentence...",Inhaler administered 10:30 – 10 puffs via spac...
1,N,A&E,Doctor,11,BD03,Pakistani,F,NKA,Winter,Morning,...,low concern,96,20,109,37,None noted,N,"Known asthmatic, has a viral illnees since yes...","Much improved since admission to A&E, now mana...","Commenced on prednisolone, is on seretide and ..."
2,N,CCDA,Doctor,3,BD04,Slovak,F,NKDA,Winter,Afternoon,...,well,96,28,140,37,None noted,N,Previously fit and well\nFamily history of ast...,No recession or signs of increased work of bre...,10 puffs 4 hourly\nHad first dose of prednisol...
3,N,GP,Doctor,3,BD06,British,M,NKDA,Winter,Afternoon,...,low concern,98,28,104,36.8,None noted,N,Rash noted to abdomen\nNo other medical history\n,Child is wheezy but is well with it. His obser...,The GP will prescribe salbutamol inhaler (he h...
4,N,GP,Doctor,3,BD09,Pakistani,M,NKA,Winter,Afternoon,...,well,97,None,None,37,None noted,N,"had an admission to LGI one year ago, gp unsur...",No wheeze at the moment but would like him to ...,"4 hourly inhaler 6 puffs, also commenced brown..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,Y,GP,Doctor,2,BD18,Other white background,M,NKA,Winter,Afternoon,...,low concern,97,35,130,38.2,None noted,Y,First wheezy episode\nStrong family history of...,"Widespread wheeze pre inhaler, improved with i...",Prednisolone- due to family history of asthma
498,Y,GP,Doctor,2,BD05,Pakistani,F,NKA,Winter,Morning,...,low concern,93,36,138,37,None noted,Y,A couple of previous wheezy episodes,Audible wheeze\nMild recession\nBilateral crep...,Amoxil and prednisolone\nSalbutamol 10 puffs v...
499,N,GP ANP,ANP,5,BD07,Indian,M,NKA,Winter,Morning,...,well,98,25,115,38,None noted,N,Known wheezer\nTakes montelukast\nPresented in...,Good air entry\nSlight wheeze\nLow grade temp\...,Prednislone to take home in case required- as ...
500,N,GP,Doctor,5,BD10,British,F,NKA,Winter,Morning,...,low concern,94,24,143,38.2,None noted,N,NaN,"Short history of cough and cold, has been coug...",20 Mcg Prednislone \n10 puffs of salbutamol 4 ...


In [34]:
import sys
sys.path.append('/Users/samrelins/Documents/LIDA/ace_project/')
from src.data_prep import *
from src.train_test import *
from IPython.display import display, HTML

data_loc = "/Users/samrelins/Documents/LIDA/ace_project/data/ace_data_orig.csv"
ace_data_orig = pd.read_csv(data_loc)

X_train, y_train, X_test, y_test = return_train_test(ace_data_orig)

best_scores, best_params = param_search_classifier(
    X_train=X_train, y_train=y_train,
    clf=LogisticRegression(random_state=0,
                           max_iter=1000,
                           solver="saga",
                           class_weight="balanced",
                           n_jobs=-1),
    param_grid={"penalty": ["none", "l1", "l2"],
                'C': np.logspace(-4, 4, 40)},
    scaled=True,
)

Required features missing to run <function add_free_text_features at 0x7fb342b6c3a0>
Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga') classifier with one_hot encoded features.


100%|██████████| 120/120 [00:47<00:00,  2.55it/s]


### Best Parameters

In [35]:
pd.DataFrame({
    "parameter": best_params.keys(),
    "value": best_params.values()
})

,parameter,value
0,C,88.862382
1,penalty,l2


### Cross Validation Scores:

In [36]:
pd.DataFrame(best_scores, index=[""]).T

,
f1,0.286667
roc_auc,0.559809
accuracy,0.661570
recall,0.407407
precision,0.221201
true_pos,7.333333
true_neg,64.333333
false_pos,26.000000
false_neg,10.666667


... and then we'll train this model on the whole training dataset and score against the test data:

In [37]:
X_train_prepped, X_test_prepped = encode_and_scale(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    cat_encoder="one_hot",
    scaled=True
)

log_reg_clf = LogisticRegression(
    random_state=0,
    max_iter=1000,
    solver="saga",
    class_weight="balanced",
    n_jobs=-1,
    **best_params
)

log_reg_clf.fit(X_train_prepped, y_train)

LogisticRegression(C=88.86238162743408, class_weight='balanced', max_iter=1000,
                   n_jobs=-1, random_state=0, solver='saga')

### Test Scores:

In [38]:
test_scores = score_classifier(log_reg_clf, X_test_prepped, y_test)
pd.DataFrame(test_scores, index=[""]).T

,
f1,0.285714
roc_auc,0.562051
accuracy,0.689441
recall,0.370370
precision,0.232558
true_pos,10.000000
true_neg,101.000000
false_pos,33.000000
false_neg,17.000000


Though it's been said before, it bears mentioning again - these results
aren't exactly great. The model is only right 64% of the time, only
identifies ~45% of the children that need to go to hospital and, of those it
predicts require hospital treatment it is correct only ~20% of the time.

These results are better than random guessing, however. If we were to guess
randomly we'd expect and accuracy and recall of around 50% with a precision
matching the proportion of positive examples in our dataset, about 16%.
As such, though the model's results aren't much better, there may be some
useful information it can provide us.

### Parameter Values

Lets take a look at the 20 largest coefficient values from our model.
Remember these are the values the model assigns each dataset feature.
Positive values indicate that the feature contributes positively to the
chance a child needs hospital treatment, and negative values the opposite.
Higher values indicate a greater contribution to the outcome, or greater
feature importance:

In [39]:
log_reg_coefs = pd.DataFrame({
    "Feature": X_train_prepped.columns,
    "Coefficient Value": log_reg_clf.coef_[0]
})
log_reg_coefs.sort_values(by="Coefficient Value", ascending=False).head(20)

,Feature,Coefficient Value
68,heart_rate,7.096439
67,resp_rate,5.439336
56,ace_heart_rate_cat_low,3.547323
35,gut_feeling_unwell,3.475496
65,age,3.185917
58,apls_resp_rate_cat_high,2.058045
61,apls_heart_rate_cat_high,1.736617
51,age_range_secondary,1.686738
6,address_bd04,1.662853
16,address_bd14,1.502937


If this looks overwhelming and hard to interpret, that's because it is - note
that these are just 20 features from a possible 80! This is one of
the limitations of this particular model. We've allowed the logistic
regression algorithm free reign to assign values to every coefficient and so it
 has done exactly that. What's worse, most of our features are "one hot"
 categories, which make it difficult to compare feature importance with
 the numerical features.

### Penalty Functions

We can, however, mitigate this confusion. If you look back at the model
parameters we originally chose from, one of these was "penalty". This is a simple addition to the cost function that penalises the model for assigning values to the
 feature coefficients / weights. The concept can be thought of simply as the
 model saying to itself:

 "I had better only assign higher weights if I'm sure they reliably contribute
  to more accurate predictions, otherwise the increased penalty value will lower
   my score"

The best performing penalty from our grid search was L2 or squared penalty,
which is formulated as follows:

$\lambda\sum_{i = 1}^{n} \beta_{i}^2$

where the $\beta{i}$'s are our coeffcients and $\lambda$ is a penalty weight
that controls the amount of penalty assigned (this was the "C" parameter when
 fitting our model).

This penalty function penalises very high weights - squared values increase
exponentially - thus makes the model less sensitive to outliers. However, it
punishes values less than one with smaller and smaller penalties as the
weight approaches zero, and so the model is allowed to assign very small
values to each weight without much issue. This has allowed the model to
 assign weights to every feature, contributing to the overwhelming collection
  we saw above.

We can, however, use a different penalty that heavily punishes **any weights
above zero** - the L1 or absolute penalty. Like the name suggests, this is a
penalty proportional to the absolute value of each weight:

$\lambda\sum_{i = 1}^{n} |\beta_{i}|$

This will encourage the model to assign values to weights **only if
 they make an important contribution to the outcome**, and otherwise keep them
 at zero. It therefore encourages the model to come up with a very sparse
 collection of weights,
 that in turn are much easier to interpret as there are less of them!

### Sparse Solution

Given that our model performed best using an L2 penalty, we may suffer a big
drop in accuracy if we switch to L1. We can check this by performing a
modified grid search to find the best logistic regression model using L1 loss:

In [40]:
l1_scores, l1_params = param_search_classifier(
    X_train=X_train, y_train=y_train,
    clf=LogisticRegression(random_state=0,
                           max_iter=1000,
                           solver="saga",
                           penalty="l1",
                           class_weight="balanced",
                           n_jobs=-1),
    param_grid={'C': np.linspace(0.01, 1, 40)},
    scaled=True,
)

Testing LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   penalty='l1', random_state=0, solver='saga') classifier with one_hot encoded features.


100%|██████████| 40/40 [00:09<00:00,  4.34it/s]


### Best L1 Penalty Parameters

In [41]:
pd.DataFrame({
    "parameter": l1_params.keys(),
    "value": l1_params.values()
})

,parameter,value
0,C,0.238462


### L1 Penalty Cross Validation Scores:

In [42]:
pd.DataFrame(l1_scores, index=[""]).T

,
f1,0.206689
roc_auc,0.488339
accuracy,0.554480
recall,0.388889
precision,0.155349
true_pos,7.000000
true_neg,53.000000
false_pos,37.333333
false_neg,11.000000


### L1 Penalty Test Scores:

In [43]:
l1_log_reg_clf = LogisticRegression(
    random_state=0,
    max_iter=1000,
    solver="saga",
    class_weight="balanced",
    n_jobs=-1,
    penalty="l1",
    **l1_params
)

l1_log_reg_clf.fit(X_train_prepped, y_train)

l1_test_scores = score_classifier(l1_log_reg_clf, X_test_prepped, y_test)
pd.DataFrame(test_scores, index=[""]).T


,
f1,0.285714
roc_auc,0.562051
accuracy,0.689441
recall,0.370370
precision,0.232558
true_pos,10.000000
true_neg,101.000000
false_pos,33.000000
false_neg,17.000000


Great - the L1 model performs only slightly worse that the best model in cross
validation and has exactly the same test set scores. The real benefit,
however, comes when checking the model coefficients:

In [44]:
param_names = X_train_prepped.columns
l1_coefs = l1_log_reg_clf.coef_[0]
nonzero_coefs = l1_coefs > 0

pd.DataFrame({
    "Feature": param_names[nonzero_coefs],
    "Coefficient Value": l1_coefs[nonzero_coefs]
}).sort_values("Coefficient Value", ascending=False)

,Feature,Coefficient Value
0,referral_from_gp,0.490367
2,apls_resp_rate_cat_high,0.288120
1,address_bd04,0.026200


Our L1 model can achieve the same results by looking at only 6 of the features!

A few things to note:
* The model estimates that all the features above contribute positively to
the outcome, that is, they each increase the probability a child requires hospital treatment.
* Both logistic regression models have selected "referral from GP" as the
categorical feature that contributes most strongly to a childs probability of
 needing hospital treatment
* As mentioned before, the magnitude / size of the coefficients indicate
their importance. However, We can't really use this rule to compare the
importance of categorical features with numerical features, or numeric features
with other numeric features, as they scale differently.
* These features are assumed by the model to operate in isolation i.e. the
model says nothing about the interactions of the different features

## Plotting Predicted Probabilities

Most importantly, it should be stressed that this model isn't very accurate.
As such, the associations it has identified between the features in the data
and the requirement for hospital treatment should be taken with a heavy pinch
 of salt. To emphasize this point, take the following plots of the models
 predicted probabilities that hospital treatment is needed, split between the
 true labels:

In [45]:
y_test_labels = y_test.apply(lambda y: "Yes" if y else "No").values
l1_prob_preds = l1_log_reg_clf.predict_proba(X_test_prepped)[:,1]

y_sort_idxs = np.argsort(y_test)
predictions_df = pd.DataFrame({
    "Hospital Required": y_test_labels[y_sort_idxs],
    "Prediction - p(Hospital Required)": l1_prob_preds[y_sort_idxs]

})

fig = px.box(predictions_df,
             y="Prediction - p(Hospital Required)",
             color="Hospital Required",
             points="all")
fig.show()

Important points to note from this plot:

* the decision boundary is 0.5 i.e. points you see above y = 0.5 were
predicted as needing hospital treatment
* the predictions for the children in each of the groups vary widely - this
is not ideal. We would really want the blue points to group towards the
bottom and the red towards the top
* there is only a slight trend for the model's predictions aligning with the true label
* the majority of children that didn't need hospital treatment have a
probability less than 0.5, but a significant minority were predicted above
* the model is very imprecise when identifying children that actually need
hospital treatment

That concludes this exploration of the logistic regression model. We'll move
on to tree models in the next notebook which will follow shortly.